In [48]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

In [49]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [50]:
df_green.createOrReplaceTempView('green')

In [51]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY 
    1,2    
""")

In [52]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7|            769.73|            45|
|2020-01-01 00:00:00|  17|195.03000000000003|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000001|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|            317.27|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [53]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [54]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [55]:
df_yellow_revenue = spark.sql(""" 
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")
df_yellow_revenue.show()


+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-12 07:00:00|   1| 42689.67999999991|          2676|
|2020-01-21 16:00:00|   1| 192422.6000000026|         11120|
|2020-01-26 17:00:00|   2|26699.479999999996|           394|
|2020-01-27 10:00:00|   2|11979.320000000003|           176|
|2020-01-18 15:00:00|   2|          21742.79|           323|
|2020-01-07 19:00:00|   2|18587.660000000007|           257|
|2020-01-16 09:00:00|   2| 8346.580000000002|           130|
|2020-01-15 09:00:00|   5|           3167.12|            62|
|2020-01-14 23:00:00|   2| 7685.450000000002|           114|
|2020-01-20 15:00:00|   2|          33292.37|           486|
|2020-01-01 07:00:00|   3|           2857.46|            33|
|2020-01-15 07:00:00|   4| 596.2399999999999|             6|
|2020-01-27 11:00:00|   5|           4424.62|            52|
|2020-01-16 05:00:00|   

In [56]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [57]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [58]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [59]:

df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [60]:

df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [61]:
df_join = spark.read.parquet('data/report/revenue/total')

In [62]:

df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [63]:
df_zones = spark.read.parquet('zones/')

In [64]:

df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [65]:

df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones', mode='overwrite')

In [66]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|NULL|              NULL|                NULL|           2215.65|                   56|
|2020-01-01 00:00:00|   1|              NULL|                NULL|234332.86000000598|                13025|
|2020-01-01 00:00:00|   5|              NULL|                NULL| 5488.679999999999|                   86|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|              NULL|                 NULL|
|2020-01-01 00:00:00|  29|              61.3|                   1|              NULL|                 NULL|
|2020-01-01 00:00:00|  33|            317.27|                  11|              NULL|                 NULL|
|2020-01-01 00:00:00|  35|  